## TODO

- get automatic emax
- warning for insulators about the fermi level positioning
- add mesh and converge to batch
- More tests
- Better docs
- dependency on how we choose the perpendicular directions
- Use ReadThe Docs [addons](https://docs.readthedocs.io/en/stable/addons.html)


In [1]:
import numpy as np

import grogupy
import grogupy.viz

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/_tqdm.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# infile = "../benchmarks/Fe_colinear/Fe.fdf"
infile = "../benchmarks/Cr3/Cr3.fdf"
# infile = "../benchmarks/Fe3GeTe2/Fe3GeTe2.fdf"

In [3]:
simulation_kspace = grogupy.Kspace([1, 1, 1])
simulation_kspace.plot().show()
simulation_kspace

<grogupy.Kspace kset=[1 1 1], NK=1>

In [4]:
simulation_contour = grogupy.Contour(
    eset=100,
    esetp=10000,
    emin=None,
    emax=0,
    emin_shift=-5,
    emax_shift=0,
    eigfile=infile,
)
simulation_contour.plot().show()
simulation_contour

<grogupy.Contour emin=-8.074511730000001, emax=0, eset=100, esetp=10000>

In [5]:
simulation_hamiltonian = grogupy.Hamiltonian(
    infile,
    [0, 0, 1],
)

Calculate H_XC: 100%|██████████| 3/3 [00:00<00:00, 7371.36it/s]


In [6]:
xyz = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

simulation = grogupy.Builder(xyz, matlabmode=True)

simulation.add_kspace(simulation_kspace)
simulation.add_contour(simulation_contour)
simulation.add_hamiltonian(simulation_hamiltonian)
simulation.ref_xcf_orientations

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/physics/builder.py:233: UserWarning:

Matlabmode is used, the exchange field reference directions were set to x,y,z!



[{'o': array([1, 0, 0]),
  'vw': [array([0, 1, 0]),
   array([0, 0, 1]),
   array([0.        , 0.70710678, 0.70710678])]},
 {'o': array([0, 1, 0]),
  'vw': [array([1, 0, 0]),
   array([0, 0, 1]),
   array([0.70710678, 0.        , 0.70710678])]},
 {'o': array([0, 0, 1]),
  'vw': [array([1, 0, 0]),
   array([0, 1, 0]),
   array([0.70710678, 0.70710678, 0.        ])]}]

In [7]:
magnetic_entities = [
    dict(atom=0, l=2),
    dict(atom=1, l=2),
    dict(atom=2, l=2),
]
simulation.add_magnetic_entities(magnetic_entities)
simulation.magnetic_entities

Add magnetic entities:: 100%|██████████| 3/3 [00:00<00:00, 320.62it/s]


[<grogupy.MagneticEntity tag=0Cr(l:2), SBS=20>,
 <grogupy.MagneticEntity tag=1Cr(l:2), SBS=20>,
 <grogupy.MagneticEntity tag=2Cr(l:2), SBS=20>]

In [8]:
pairs = [
    dict(ai=0, aj=1, Ruc=np.array([0, 0, 0])),
    dict(ai=1, aj=2, Ruc=np.array([0, 0, 0])),
    dict(ai=2, aj=0, Ruc=np.array([0, 0, 0])),
]
simulation.add_pairs(pairs)
simulation.pairs

Add pairs:: 100%|██████████| 3/3 [00:00<00:00, 1796.53it/s]


[<grogupy.Pair tag1=0Cr(l:2), tag2=1Cr(l:2), Ruc=[0 0 0]>,
 <grogupy.Pair tag1=1Cr(l:2), tag2=2Cr(l:2), Ruc=[0 0 0]>,
 <grogupy.Pair tag1=2Cr(l:2), tag2=0Cr(l:2), Ruc=[0 0 0]>]

In [9]:
simulation.solve()

Setup perturbations for rotated hamiltonian: 100%|██████████| 3/3 [00:00<00:00, 20426.81it/s]


In [10]:
print(simulation.to_magnopy())

# ================================================================================
# grogupy version: 0.0.7
# Input file: ../benchmarks/Cr3/Cr3.fdf
# Spin mode: SPIN-ORBIT
# ================================================================================
# SLURM job ID: Could not be determined.
# Architecture: CPU
# Number of nodes in the parallel cluster: 1
# Parallelization is over: K
# Solver used for Greens function calculation: Sequential
# Solver used for Exchange tensor: grogupy
# Solver used for Anisotropy tensor: grogupy
# ================================================================================
# Cell [Ang]:
# 1.442498074906033700e+01 -2.498479955557547072e+01 0.000000000000000000e+00
# 1.442498074906033700e+01 2.498479955557547072e+01 0.000000000000000000e+00
# 0.000000000000000000e+00 0.000000000000000000e+00 2.884996149812067401e+01
# ================================================================================
# DFT axis: [0 0 1]
# Quantization axis and perpendi

In [11]:
K_fit = []
K_calc = []
K_cons = []
for mag_ent in simulation.magnetic_entities:
    mag_ent.fit_anisotropy_tensor(simulation.ref_xcf_orientations)
    K_fit.append(mag_ent.K_meV)
    mag_ent.calculate_anisotropy()
    K_calc.append(mag_ent.K_meV)
    K_cons.append(mag_ent.K_consistency_meV)

K_fit = np.array(K_fit)
K_calc = np.array(K_calc)


J_fit = []
D_fit = []
S_fit = []
J_calc = []
D_calc = []
S_calc = []
for pair in simulation.pairs:
    pair.fit_exchange_tensor(simulation.ref_xcf_orientations)
    J_fit.append(pair.J_meV)
    D_fit.append(pair.D_meV)
    S_fit.append(pair.J_S_meV)
    pair.calculate_exchange_tensor()
    J_calc.append(pair.J_meV)
    D_calc.append(pair.D_meV)
    S_calc.append(pair.J_S_meV)

J_fit = np.array(J_fit)
J_calc = np.array(J_calc)
D_fit = np.array(D_fit)
D_calc = np.array(D_calc)
S_fit = np.array(S_fit)
S_calc = np.array(S_calc)

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/physics/utilities.py:534: UserWarning:

This is experimenal!

/Users/danielpozsar/Documents/studies/elte/phd/grogu/.venv/lib/python3.12/site-packages/grogupy/physics/utilities.py:676: UserWarning:

This is experimenal!



In [12]:
xyz = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

simulation_2 = grogupy.Builder(xyz, matlabmode=False)

simulation_2.add_kspace(simulation_kspace)
simulation_2.add_contour(simulation_contour)
simulation_2.add_hamiltonian(simulation_hamiltonian)
simulation_2.add_magnetic_entities(magnetic_entities)
simulation_2.add_pairs(pairs)
simulation_2.pairs
simulation_2.solve()

J_fit_2 = []
D_fit_2 = []
S_fit_2 = []
for pair in simulation_2.pairs:
    pair.fit_exchange_tensor(simulation_2.ref_xcf_orientations)
    J_fit_2.append(pair.J_meV)
    D_fit_2.append(pair.D_meV)
    S_fit_2.append(pair.J_S_meV)

J_fit_2 = np.array(J_fit_2)
D_fit_2 = np.array(D_fit_2)
S_fit_2 = np.array(S_fit_2)

K_fit_2 = []
for mag_ent in simulation_2.magnetic_entities:
    mag_ent.fit_anisotropy_tensor(simulation_2.ref_xcf_orientations)
    K_fit_2.append(mag_ent.K_meV)
K_fit_2 = np.array(K_fit_2)

Setup perturbations for rotated hamiltonian: 100%|██████████| 3/3 [00:00<00:00, 21254.92it/s]


In [13]:
xyz = np.array(
    [
        [np.cos(np.pi / 3), np.sin(np.pi / 3), 0],
        [np.cos(2 * np.pi / 3), np.sin(2 * np.pi / 3), 0],
        [1, 0, 0],
        # [0, 1, 0],
        # [0, 0, 1],
    ]
)

simulation_3 = grogupy.Builder(xyz)

simulation_3.add_kspace(simulation_kspace)
simulation_3.add_contour(simulation_contour)
simulation_3.add_hamiltonian(simulation_hamiltonian)
simulation_3.add_magnetic_entities(magnetic_entities)
simulation_3.add_pairs(pairs)
simulation_3.solve()

J_fit_3 = []
D_fit_3 = []
S_fit_3 = []
for pair in simulation_3.pairs:
    pair.fit_exchange_tensor(simulation_3.ref_xcf_orientations)
    J_fit_3.append(pair.J_meV)
    D_fit_3.append(pair.D_meV)
    S_fit_3.append(pair.J_S_meV)

J_fit_3 = np.array(J_fit_3)
D_fit_3 = np.array(D_fit_3)
S_fit_3 = np.array(S_fit_3)

K_fit_3 = []
for mag_ent in simulation_3.magnetic_entities:
    mag_ent.fit_anisotropy_tensor(simulation_3.ref_xcf_orientations)
    K_fit_3.append(mag_ent.K_meV)
K_fit_3 = np.array(K_fit_3)

Setup perturbations for rotated hamiltonian: 100%|██████████| 3/3 [00:00<00:00, 31615.36it/s]


In [14]:
rounding = 3
K_cons = np.around(K_cons, rounding)

J_calc = np.around(J_calc, rounding)
D_calc = np.around(D_calc, rounding)
S_calc = np.around(S_calc, rounding)
K_calc = np.around(K_calc, rounding)


J_fit = np.around(J_fit, rounding)
D_fit = np.around(D_fit, rounding)
S_fit = np.around(S_fit, rounding)
K_fit = np.around(K_fit, rounding)

J_fit_2 = np.around(J_fit_2, rounding)
D_fit_2 = np.around(D_fit_2, rounding)
S_fit_2 = np.around(S_fit_2, rounding)
K_fit_2 = np.around(K_fit_2, rounding)

J_fit_3 = np.around(J_fit_3, rounding)
D_fit_3 = np.around(D_fit_3, rounding)
S_fit_3 = np.around(S_fit_3, rounding)
K_fit_3 = np.around(K_fit_3, rounding)

In [15]:
print("K_cons should be zero")
K_cons

K_cons should be zero


array([0.085, 0.085, 0.17 ])

In [16]:
print("J_fit, J_calc")
print((J_fit == J_calc).all())
print("K_fit, K_calc")
print((K_fit == K_calc).all())
print()
print("J_fit, J_fit2")
print((J_fit == J_fit_2).all())
print("K_fit, K_fit2")
print((K_fit == K_fit_2).all())
print()
print("J_fit, J_fit3")
print((J_fit == J_fit_3).all())
print("K_fit, K_fit3")
print((K_fit == K_fit_3).all())

J_fit, J_calc
False
K_fit, K_calc
False

J_fit, J_fit2
True
K_fit, K_fit2
True

J_fit, J_fit3
False
K_fit, K_fit3
False


In [17]:
print("Off diagonal part is equal")
print("D_fit, D_calc")
print((D_fit == D_calc).all())
print()
print("S_fit, S_calc")
print((S_fit == S_calc).all())
print()
print("Jfit, Jcalc")
print(J_fit[:, 0, 0])
print(J_calc[:, 0, 0])
print()
print(J_fit[:, 1, 1])
print(J_calc[:, 1, 1])
print()
print(J_fit[:, 2, 2])
print(J_calc[:, 2, 2])

Off diagonal part is equal
D_fit, D_calc
True

S_fit, S_calc
True

Jfit, Jcalc
[148.038 148.139 148.139]
[149.181 149.317 149.317]

[148.173 148.071 148.071]
[146.983 146.917 146.917]

[149.176 149.176 149.176]
[149.181 149.173 149.173]


In [18]:
print("K_fit, K_calc")
print(K_fit[:, 0, 0])
print(K_calc[:, 0, 0])
print()
print(K_fit[:, 1, 1])
print(K_calc[:, 1, 1])
print()
print(K_fit[:, 0, 1])
print(K_calc[:, 0, 1])
print()
print(K_fit[:, 0, 2])
print(K_calc[:, 0, 2])
print()
print(K_fit[:, 1, 2])
print(K_calc[:, 1, 2])

K_fit, K_calc
[-0.144 -0.144 -0.113]
[-0.116 -0.116 -0.169]

[-0.123 -0.123 -0.155]
[-0.152 -0.152 -0.098]

[ 0.042 -0.042  0.   ]
[ 0.043 -0.043  0.   ]

[ 0.002 -0.002 -0.   ]
[ 0.001 -0.001 -0.   ]

[ 0.002 -0.002 -0.   ]
[ 0.001 -0.001 -0.   ]


In [19]:
print("Off diagonal part is equal")
print("D_fit, D_fit_3")
print((D_fit == D_fit_3).all())
print()
print("S_fit, S_fit_3")
print((S_fit == S_fit_3).all())
print()
print("J_fit, J_fit_3")
print(J_fit[:, 0, 0])
print(J_fit_3[:, 0, 0])
print()
print(J_fit[:, 1, 1])
print(J_fit_3[:, 1, 1])
print()
print(J_fit[:, 2, 2])
print(J_fit_3[:, 2, 2])

Off diagonal part is equal
D_fit, D_fit_3
False

S_fit, S_fit_3
False

J_fit, J_fit_3
[148.038 148.139 148.139]
[-2045.355 -1487.242 -1895.59 ]

[148.173 148.071 148.071]
[880.875 694.745 830.862]

[149.176 149.176 149.176]
[149.176 149.176 149.176]


In [20]:
print("K_fit, K_fit_3")
print(K_fit[:, 0, 0])
print(K_fit_3[:, 0, 0])
print()
print(K_fit[:, 1, 1])
print(K_fit_3[:, 1, 1])
print()
print(K_fit[:, 0, 1])
print(K_fit_3[:, 0, 1])
print()
print(K_fit[:, 0, 2])
print(K_fit_3[:, 0, 2])
print()
print(K_fit[:, 1, 2])
print(K_fit_3[:, 1, 2])

K_fit, K_fit_3
[-0.144 -0.144 -0.113]
[0.187 0.184 0.188]

[-0.123 -0.123 -0.155]
[-0.152 -0.152 -0.098]

[ 0.042 -0.042  0.   ]
[-0.021  0.021  0.   ]

[ 0.002 -0.002 -0.   ]
[ 0.01 -0.01 -0.  ]

[ 0.002 -0.002 -0.   ]
[0.044 0.042 0.053]


In [21]:
from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_rotvec(np.pi / 2 * np.array([0, 0, 1]))
r.as_matrix()

array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [22]:
grogupy.save(simulation, "test_builder.pkl", compress=0)
grogupy.save(simulation.contour, "test_contour.pkl", compress=0)
grogupy.save(simulation.kspace, "test_kspace.pkl", compress=0)
grogupy.save(simulation.hamiltonian, "test_hamiltonian.pkl", compress=0)
grogupy.save(simulation.pairs[0], "test_pair.pkl", compress=0)
grogupy.save(simulation.magnetic_entities[0], "test_magnetic_entity.pkl", compress=0)

In [23]:
grogupy.save_UppASD(simulation, "./")